In [2]:

import zipfile
import os
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import requests
import xml.etree.ElementTree as ET
import tensorflow as t
import torch
import torch.nn as nn

2024-06-26 20:01:17.840718: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 20:01:19.555747: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [23]:
import os
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda-12.3'
#is supposed to make Sequential model work

# Unzipping csv file

In [3]:

uploaded_zip = '/home/claney/EclipsePlatform/eclipse_platform.zip'
extract_dir = '/home/claney/extracted'
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir)
with zipfile.ZipFile(uploaded_zip, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

csv_file = os.path.join(extract_dir, 'eclipse_platform.csv')
full_df = pd.read_csv(csv_file)

# Spliting testing and training

In [4]:
length=len(full_df)
split_index=int(.8*length)
df=full_df.iloc[:split_index] #df=training set

# Fill NAN

In [5]:
full_df.fillna(0,inplace=True)

# Creating full_text_data column

In [37]:
#creating freeform textual data column
df['full_text_data']=df['Description']+df['Title']
print(df['full_text_data'])
df['full_text_data']=df['full_text_data'].fillna('')

0        - Setup a project that contains a *.gif resour...
1        Opening repository resource always open the de...
2        KM (10/2/2001 5:55:18 PM); \tThis PR about the...
3        - become synchronized with some project in a r...
4        For getting/setting the managed state of a res...
                               ...                        
68119    Build ID: 3.3 (I am not sure what You mean by ...
68120    There are several NPEs within the UI session t...
68121    I20080501-0100; ; Now that were using the coll...
68122    Created attachment 98318; screenshot; ; I20080...
68123    Build ID: M20080221-1800; ; Steps To Reproduce...
Name: full_text_data, Length: 68124, dtype: object


/tmp/ipykernel_173824/3345744263.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text_data']=df['Description']+df['Title']
/tmp/ipykernel_173824/3345744263.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['full_text_data']=df['full_text_data'].fillna('')


# Creating isDuplicate column

In [6]:
def is_duplicate(val):
    return 1 if val !=0.0 else 0
df['isDuplicate'] = df['Duplicated_issue'].apply(lambda x: is_duplicate(x))

print(df['isDuplicate'])

0        0
1        0
2        0
3        0
4        0
        ..
68119    0
68120    0
68121    0
68122    0
68123    0
Name: isDuplicate, Length: 68124, dtype: int64


/tmp/ipykernel_173824/1537637410.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['isDuplicate'] = df['Duplicated_issue'].apply(lambda x: is_duplicate(x))


# Splitting training into training and testing

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = df['Description']  # Input features
y = df['isDuplicate']       # Target variable

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# GPU setup

In [ ]:
# Check for CUDA availability
if torch.cuda.is_available():
    print("CUDA is available. Number of GPUs:", torch.cuda.device_count())
    print("CUDA device name:", torch.cuda.get_device_name(4))
else:
    print("CUDA is not available.")
torch.cuda.set_device(torch.device("cuda:4"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


## MLP neural network

In [11]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


In [13]:


# Vectorize the text data
vectorizer = TfidfVectorizer(max_features=1000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# neural net
model1 = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', max_iter=1000)
model1.to(device)
model1.fit(X_train_vec, y_train)

# Make predictions on the test set
y_pred = model1.predict(X_test_vec)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.7413540015266279


# Sequential model

In [18]:
pip install tensorflow-gpu

  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [44 lines of output]
      Traceback (most recent call last):
        File "/home/claney/anaconda3/envs/reu/lib/python3.11/site-packages/setuptools/_vendor/packaging/requirements.py", line 35, in __init__
          parsed = _parse_requirement(requirement_string)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/home/claney/anaconda3/envs/reu/lib/python3.11/site-packages/setuptools/_vendor/packaging/_parser.py", line 64, in parse_requirement
          return _parse_requirement(Tokenizer(source, rules=DEFAULT_RULES))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/home/claney/anaconda3/envs/reu/lib/python3.11/site-packages/setuptools/_vendor/packaging/_parser.py", line 82, in _parse_requirement
          url, specifier, marker = _parse_requirement_details(tokenizer

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.layers import Embedding
# Tokenize the text descriptions
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad the sequences to ensure uniform input size
max_sequence_length = 100
X_train_pad = pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Build the Sequential model
model2 = Sequential()
model2.add(Embedding(input_dim=5000, output_dim=64))
model2.add(LSTM(64, return_sequences=True))
model2.add(Dropout(0.5))
model2.add(LSTM(32))
model2.add(Dense(1, activation='sigmoid'))

# Compile the model
model2.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model2.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model2.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {accuracy:.4f}')


Epoch 1/10


2024-06-26 20:32:12.986019: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.
2024-06-26 20:32:12.986090: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: UNKNOWN: JIT compilation failed.
	 [[{{function_node __inference_one_step_on_data_10726}}{{node adam/Pow_13}}]]
2024-06-26 20:32:12.986115: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous send item cancelled. Key hash: 8450978442353663687
2024-06-26 20:32:12.986145: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.
2024-06-26 20:32:12.986178: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.
2024-06-26 20:32:12.986209: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.
2024-06-26 20:32:12.986229: W tensorflow/core/framework/op_kernel.cc:1827] UNKNOWN: JIT compilation failed.
2024-06-26 20:32:12.986248: W tensorflow/core/framework/op_kernel.cc:1827] U

UnknownError: Graph execution error:

Detected at node adam/Pow_13 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/claney/anaconda3/envs/reu/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/claney/anaconda3/envs/reu/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/claney/anaconda3/envs/reu/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/claney/anaconda3/envs/reu/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/claney/anaconda3/envs/reu/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/home/claney/anaconda3/envs/reu/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/home/claney/anaconda3/envs/reu/lib/python3.11/asyncio/events.py", line 84, in _run

  Fi

# BERT trasformer model

In [ ]:
#gave thiis error
#RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
#Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [20]:
pip install tf-keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.1 MB/s eta 0:00:00:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [29]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Assuming you have your data prepared as X_train, y_train, X_test, y_test

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model3 = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Tokenize inputs
def tokenize_text(text):
    return tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        return_attention_mask=True,
        truncation=True
    )

# Tokenize and encode inputs
X_train_tokens = [tokenize_text(text) for text in X_train]
X_test_tokens = [tokenize_text(text) for text in X_test]

# Convert tokens to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((
    {key: tf.constant([token[key] for token in X_train_tokens]) for key in X_train_tokens[0].keys()},
    tf.constant(y_train)
)).shuffle(100).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {key: tf.constant([token[key] for token in X_test_tokens]) for key in X_test_tokens[0].keys()},
    tf.constant(y_test)
)).batch(32)

# Fine-tune BERT model
var_name = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name='adam',
)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model3.compile(optimizer=var_name, loss=loss, metrics=['accuracy'])

# Train the model
model3.fit(train_dataset, epochs=3, validation_data=test_dataset)

# Evaluate the model
results = model3.evaluate(test_dataset)
print(f'Loss: {results[0]}, Accuracy: {results[1]}')



All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x7fd0fc331910>

# UGhHH im trying anything at this point

In [34]:
pip install transformers tensorflow


Note: you may need to restart the kernel to use updated packages.


In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

# Sample data (replace with your own bug reports and labels)


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 20  # Max length of your sequences

# Tokenize and encode sequences
encoded_data = tokenizer(
    df['full_text_data'].tolist(),
    padding=True,
    truncation=True,
    max_length=max_len,
    return_tensors='tf'
)

labels = np.array(df['isDuplicate'])

# Convert tensors to numpy arrays
input_ids_np = np.array(encoded_data['input_ids'])
attention_mask_np = np.array(encoded_data['attention_mask'])

# Split into training and test sets
X_train_ids, X_test_ids, X_train_masks, X_test_masks, y_train, y_test = train_test_split(
    input_ids_np,
    attention_mask_np,
    labels,
    test_size=0.2,
    random_state=42
)



In [40]:
from transformers import TFBertForSequenceClassification
import tensorflow as tf

# Load pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=3,
    batch_size=32,
    validation_data=(X_test, y_test)
)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x7fd1247a6b10>